In [6]:
import SimpleITK as sitk
import math
from pyquaternion import Quaternion
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from nilearn import plotting, image, datasets
%matplotlib inline

/home/ltetrel/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def transform_volume(transf, ref_grid, interp, brain):
    def_pix = 0.0
    brain_to_grid = sitk.Resample(
        brain, ref_grid, transf, interp, def_pix, sitk.sitkFloat32)

    return brain_to_grid

def create_ref_grid(target_brain=None):

    if target_brain is None:
        # Then, it will be a grid near the MNI152 template
        spacing = (1., 1., 1.)
        direction = (1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
        size = (250, 250, 250)
        pixel_type = 8
        origin = np.array([-89, 125, -71])
        t = np.array([182., 218., 182.])
    else:
        # pixel spacing (mm), lower space to have more pixel in a given area
        spacing = tuple(np.array(target_brain.GetSpacing()) / 2)
        direction = target_brain.GetDirection()
        size = tuple(3 * [int(max(target_brain.GetSize()) * 2.4)])
        pixel_type = target_brain.GetPixelIDValue()
        origin = target_brain.GetOrigin()
        t = np.array(target_brain.GetSpacing()) * np.array(target_brain.GetSize())

    # we want the grid to have the same center as target
    t = (t - np.array(spacing) * np.array(size)) / 2
    # pixel (0,0,0) is at top left
    origin = origin + t * np.array([1., -1., 1.])

    # construction of the reference
    ref_grid = sitk.Image(size, pixel_type)
    ref_grid.SetOrigin(origin)
    ref_grid.SetSpacing(spacing)
    ref_grid.SetDirection(direction)

    return ref_grid

In [3]:
input_path = "/home/ltetrel/Documents/data/neuromod/to_generate/ses-game001_task-shinobi1_run-01_bold.nii.gz"
output_path_fixed = "/home/ltetrel/Documents/data/neuromod/to_generate/fixed.nii.gz"
output_path = "/home/ltetrel/Documents/data/neuromod/to_generate/moving.nii.gz"

source_brain = sitk.ReadImage(input_path, sitk.sitkFloat32)
ref_grid = create_ref_grid()

In [4]:
extract_size = np.array(source_brain.GetSize())
idx = math.floor(extract_size[3] / 2)
extract_size[3] = 0
extract = sitk.ExtractImageFilter()
extract.SetIndex([0, 0, 0, idx])
extract.SetSize(extract_size.tolist())
fixed_brain = extract.Execute(source_brain)
sitk.WriteImage(fixed_brain, output_path_fixed)

In [21]:
translation = sitk.TranslationTransform(3, tuple([0,0,0]))
theta = -math.pi/4
axis = [-1, 0, 0]
q = [np.cos(theta/2), axis[0]*np.sin(theta/2), axis[1]*np.sin(theta/2), axis[2]*np.sin(theta/2)]
# itk defines quaternion as [q1, q2, q3, q0]
rigid = sitk.VersorRigid3DTransform([q[1], q[2], q[3], q[0]])
rigid.SetTranslation(translation.GetOffset())

moving_brain = transform_volume(rigid, ref_grid, sitk.sitkBSplineResampler, fixed_brain)
sitk.WriteImage(moving_brain, output_path)

In [15]:
plotting.view_img(output_path_fixed)

In [8]:
plotting.view_img(output_path)